In [1]:
# Imports necessários
import os
import PIL
import math
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory

from IPython.display import display

2021-08-23 20:25:14.549338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-23 20:25:14.549439: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Pasta com imgs
imgs: str = r"/home/dangelo/Downloads/imgs_teste"
os.listdir(imgs)

['img20210515_17195465.jpg',
 'img20210515_16403543.jpg',
 'img20210515_16385031.jpg',
 '45.jpg',
 'img20210515_15373823.jpg',
 'img20210515_17374084.jpg',
 'img20210515_16464084.jpg',
 'img20210515_16425192.jpg',
 'img20210515_17472074.jpg',
 'img20210515_16360937.jpg',
 'img20210515_15595523.jpg',
 'img20210515_16040522.jpg',
 'img20210515_17253306.jpg',
 'img20210515_16482898.jpg',
 'img20210515_17571504.jpg',
 'img20210515_15540198.jpg',
 'img20210515_15351416.jpg',
 'img20210515_16374742.jpg',
 'img20210515_17311878.jpg',
 'img20210515_15584433.jpg',
 'img20210515_16075108.jpg',
 'img20210515_15570053.jpg',
 'img20210515_17043668.jpg',
 'img20210515_16501795.jpg',
 'img20210515_16005061.jpg',
 '5.jpg',
 '85.jpg',
 'img20210515_16022920.jpg',
 'img20210515_16102331.jpg',
 '12.jpg',
 'img20210515_16394287.jpg',
 'img20210515_16513647.jpg',
 '1.jpg',
 'img20210515_17505084.jpg',
 'img20210515_17053501.jpg',
 'img20210515_16565564.jpg',
 '8575.jpg',
 'img20210515_17013073.jpg',
 'img2

In [5]:
contador = 1

for img in os.listdir(imgs):
    caminho_total = os.path.join(imgs, img)
    novo_caminho = os.path.join(imgs, f"img_{contador}_lr.jpg")
    os.rename(caminho_total, novo_caminho)
    contador += 1

In [8]:
def constroi_modelo(upscale_factor=3, channels=1):
    conv_args = {
        "activation": "relu",
        "kernel_initializer": "Orthogonal",
        "padding": "same",
    }
    inputs = keras.Input(shape=(None, None, channels))
    x = layers.Conv2D(64, 5, **conv_args)(inputs)
    x = layers.Conv2D(64, 3, **conv_args)(x)
    x = layers.Conv2D(32, 3, **conv_args)(x)
    x = layers.Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)
    
    model = keras.Model(inputs, outputs)
    
    loss_fn = keras.losses.MeanSquaredError()
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss=loss_fn,
    )
    return model


def conversor_alta_resolucao(modelo, img):
    ycbcr = img.convert("YCbCr")
    img_cinza, canal_cb, canal_cr = ycbcr.split()
    img_cinza = img_to_array(img_cinza)
    img_cinza = img_cinza.astype("float32") / 255.0

    entrada = np.expand_dims(img_cinza, axis=0)
    saida = modelo.predict(entrada)

    saida_img_cinza = saida[0]
    saida_img_cinza *= 255.0

    saida_img_cinza = saida_img_cinza.clip(0, 255)
    saida_img_cinza = saida_img_cinza.reshape(
        (np.shape(saida_img_cinza)[0], 
         np.shape(saida_img_cinza)[1])
    )

    saida_img_cinza = PIL.Image.fromarray(np.uint8(saida_img_cinza), mode="L")
    saida_canal_cb = canal_cb.resize(saida_img_cinza.size, PIL.Image.BICUBIC)
    saida_canal_cr = canal_cr.resize(saida_img_cinza.size, PIL.Image.BICUBIC)
    img_resultado = PIL.Image.merge("YCbCr", 
            (saida_img_cinza, saida_canal_cb, saida_canal_cr)).convert(
        "RGB"
    )
    return img_resultado


def abreImagem(nome):
    return PIL.Image.open(nome)


def printImagem(modelo, nome):
    imgBaixa = abreImagem(nome)
    display(imgBaixa)
    
    imgAlta = conversor_alta_resolucao(modelo, imgBaixa)
    display(imgAlta)
    
    print(f"Imagem baixa: {imgBaixa.size[0], imgBaixa.size[1]}")
    print(f"Imagem alta: {imgAlta.size[0], imgAlta.size[1]}")
    

# Pasta com os checkpoints
PASTA_CHECKPOINT: str = r"/home/dangelo/dev/TCC/image-resolution/src/criacao_modelo/checkpoint_100_epocas"
ARQUIVO_CHECKPOINT: str = os.path.join(PASTA_CHECKPOINT, "meu-checkpoint")

In [7]:
# Construindo o modelo
modelo = constroi_modelo(upscale_factor=3, channels=1)
modelo.load_weights(ARQUIVO_CHECKPOINT)
modelo.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 32)    18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 9)     2601      
_________________________________________________________________
tf.nn.depth_to_space (TFOpLa (None, None, None, 1)     0         
Total params: 59,657
Trainable params: 59,657
Non-trainable params: 0
_________________________________________________________

2021-08-23 20:31:57.557440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-23 20:31:57.557466: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-23 20:31:57.557495: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dangelo-mint): /proc/driver/nvidia/version does not exist
2021-08-23 20:31:57.557764: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
for img in lista_imagens:
    caminho_img = os.path.join(imgs, img)
    imagem_lr = abreImagem(caminho_img)
    
    
    caminho_img_hr = caminho_img.replace("_lr", "_hr")
    imagem_hr = conversor_alta_resolucao(modelo, imagem_lr)
    
    imagem_hr.save(caminho_img_hr)
    
    contador += 1